In [21]:
#Fav / Dog Win percentage + odds filter
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
stake=100
datefilter='2023-01-01'

devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()

def hist_data_dog(surface, sex, odds_low,odds_high,date):
    data=pd.read_sql_query(f"Select distinct * from results_{surface}_1 where date > '{date}' ",con=devengine)#.to_clipboard(index=False)
    data["Fav_Odds"] = data.Fav_Odds.astype(float)
    data["Dog_Odds"] = data.Dog_Odds.astype(float)
    data['Fav_Win'] = data['Winner']==data['Fav']
    data['DogWin'] = data['Winner']!=data['Fav']
    data=data[(data['Fav_Odds']>=odds_low)&(data['Fav_Odds']<=odds_high)]
    data=data[data['Sex']==sex]
    return data
yo=pd.read_sql_query(f"Select DISTINCT * From Elo_AllMatches_Hard",con=devengine)#.to_clipboard(index=False)


In [22]:
clay_womens=yo[yo['Sex']=='Womens']#hist_data_dog('grass','Womens',1,2,datefilter)
clay_mens=yo[yo['Sex']=='Mens']
temp=pd.concat([clay_womens,clay_mens]).sort_values(by='Date')
temp = temp.reset_index(drop=True)
temp['Fav_Win']=(temp['Winner']==temp['Fav']).astype(int)

temp['Dog_Win']=(temp['Winner']==temp['Dog']).astype(int)
temp["Fav_Odds"] = temp.Fav_Odds.astype(float)
temp["Dog_Odds"] = temp.Dog_Odds.astype(float)
temp

,Surface,Date,Sex,Winner,Loser,Winner_Odds,Loser_Odds,Resulted,Time,Prob_Elo,...,Fav,Dog,Fav_Odds,Dog_Odds,Fav_Rank,Dog_Rank,Fav_Rank_High,Dog_Rank_High,Fav_Win,Dog_Win
0,Hard,2023-07-07,Mens,Tristan Schoolkate,Altug Celikbilek,2.36,1.55,None,None,0.500000,...,Altug Celikbilek,Tristan Schoolkate,1.55,2.36,211.0,326.0,NaN,NaN,0,1
1,Hard,2023-07-07,Mens,Steve Johnson,James McCabe,1.6,2.23,None,None,0.500000,...,Steve Johnson,James McCabe,1.60,2.23,219.0,277.0,NaN,NaN,1,0
2,Hard,2023-07-07,Mens,Omar Jasika,Yuta Shimizu,1.71,2.06,None,None,0.500000,...,Omar Jasika,Yuta Shimizu,1.71,2.06,286.0,252.0,NaN,NaN,1,0
3,Hard,2023-07-08,Mens,Tennys Sandgren,Sasi Kumar Mukund,1.31,3.32,None,None,0.429040,...,Tennys Sandgren,Sasi Kumar Mukund,1.31,3.32,240.0,385.0,NaN,NaN,1,0
4,Hard,2023-07-08,Mens,Steve Johnson,Yasutaka Uchiyama,1.79,1.97,None,None,0.586958,...,Steve Johnson,Yasutaka Uchiyama,1.79,1.97,219.0,267.0,NaN,NaN,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15215,Hard,2025-07-04,Mens,Tristan Schoolkate,Andre Ilagan,1.22,3.92,None,None,0.595332,...,Tristan Schoolkate,Andre Ilagan,1.22,3.92,104.0,371.0,102.0,362.0,1,0
15216,Hard,2025-07-05,Mens,Yibing Wu,Michael Zheng,1.61,2.23,True,07:10,0.671976,...,Yibing Wu,Michael Zheng,1.61,2.23,285.0,498.0,54.0,498.0,1,0
15217,Hard,2025-07-05,Mens,Rei Sakamoto,Tristan Schoolkate,2.29,1.57,True,02:10,0.331852,...,Tristan Schoolkate,Rei Sakamoto,1.57,2.29,104.0,291.0,102.0,278.0,0,1
15218,Hard,2025-07-05,Mens,Patrick Kypson,Alexis Galarneau,1.41,2.75,True,04:25,0.585080,...,Patrick Kypson,Alexis Galarneau,1.41,2.75,240.0,207.0,133.0,153.0,1,0


In [23]:
import pandas as pd
calc=temp.copy()
# Ensure 'Date' is a datetime type
calc['Date'] = pd.to_datetime(calc['Date'])

# Melt Fav and Dog results into long format for players and their wins
fav_df = calc[['Date', 'Fav', 'Fav_Win']].rename(columns={'Fav': 'Player', 'Fav_Win': 'Win'})
dog_df = calc[['Date', 'Dog', 'Dog_Win']].rename(columns={'Dog': 'Player', 'Dog_Win': 'Win'})

# Concatenate both datasets
player_results = pd.concat([fav_df, dog_df], ignore_index=True)

# Keep only the rows where there was a win
player_wins = player_results[player_results['Win'] == True].copy()

# Sort player wins by Player and Date
player_wins = player_wins.sort_values(['Player', 'Date'])

# Function to count wins in the next 30 days after the match (excludes current match)
def wins_last_30_after_match(group):
    result = []
    for i in range(len(group)):
        cutoff = group.iloc[i]['Date']
        # Filter for wins that happened after the cutoff date within the next 30 days
        future_wins = group[(group['Date'] > cutoff) & (group['Date'] <= cutoff + pd.Timedelta(days=30))]
        result.append(len(future_wins))  # Count the wins in the next 30 days
    return pd.Series(result, index=group.index)

# Calculate wins in the next 30 days after each match
player_wins['Wins_Last_30_Days_After'] = (
    player_wins.groupby('Player', group_keys=False)
    .apply(wins_last_30_after_match)
)

# Merge the calculated win counts back into the original dataframe

# Sort for merge_asof (allows for the "after match" logic)
calc = calc.sort_values('Date')
player_wins = player_wins.sort_values('Date')

# Merge back for Fav (after the match date)
fav_win_counts = player_wins[['Player', 'Date', 'Wins_Last_30_Days_After']].rename(columns={
    'Player': 'Fav',
    'Date': 'Match_Date',
    'Wins_Last_30_Days_After': 'Fav_Wins_Last_30_Days_After'
})

calc = pd.merge_asof(
    calc.sort_values('Date'),
    fav_win_counts.sort_values('Match_Date'),
    by='Fav',
    left_on='Date',
    right_on='Match_Date',
    direction='forward'
).drop(columns=['Match_Date'])

# Merge back for Dog (after the match date)
dog_win_counts = player_wins[['Player', 'Date', 'Wins_Last_30_Days_After']].rename(columns={
    'Player': 'Dog',
    'Date': 'Match_Date',
    'Wins_Last_30_Days_After': 'Dog_Wins_Last_30_Days_After'
})

calc = pd.merge_asof(
    calc.sort_values('Date'),
    dog_win_counts.sort_values('Match_Date'),
    by='Dog',
    left_on='Date',
    right_on='Match_Date',
    direction='forward'
).drop(columns=['Match_Date'])

# Fill any missing win counts with 0 (if no future wins)
calc['Fav_Wins_Last_30_Days_After'] = calc['Fav_Wins_Last_30_Days_After'].fillna(0).astype(int)
calc['Dog_Wins_Last_30_Days_After'] = calc['Dog_Wins_Last_30_Days_After'].fillna(0).astype(int)

Fav_Win_Count=calc[['Date','Fav','Fav_Wins_Last_30_Days_After']].copy()
Fav_Win_Count['Date'] = pd.to_datetime(Fav_Win_Count['Date'])
latest_per_fav = Fav_Win_Count[Fav_Win_Count['Date'] == Fav_Win_Count.groupby('Fav')['Date'].transform('max')]
latest_per_fav[['Fav','Fav_Wins_Last_30_Days_After']].to_csv('Fav_Win_Count_Grass.csv')
Dog_Win_Count=calc[['Date','Dog','Dog_Wins_Last_30_Days_After']].copy()
Dog_Win_Count['Date'] = pd.to_datetime(Dog_Win_Count['Date'])
latest_per_dog = Dog_Win_Count[Dog_Win_Count['Date'] == Dog_Win_Count.groupby('Dog')['Date'].transform('max')]
latest_per_dog[['Dog','Dog_Wins_Last_30_Days_After']].to_csv('Dog_Win_Count_Grass.csv')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_8172\908341263.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(wins_last_30_after_match)


In [24]:
calc=temp.copy()
import pandas as pd

# Ensure 'Date' is a datetime type
calc['Date'] = pd.to_datetime(calc['Date'])

# Melt Fav and Dog results into long format for players and their wins
fav_df = calc[['Date', 'Fav', 'Fav_Win']].rename(columns={'Fav': 'Player', 'Fav_Win': 'Win'})
dog_df = calc[['Date', 'Dog', 'Dog_Win']].rename(columns={'Dog': 'Player', 'Dog_Win': 'Win'})

# Concatenate both datasets
player_results = pd.concat([fav_df, dog_df], ignore_index=True)

# Keep only the rows where there was a win
player_wins = player_results[player_results['Win'] == True].copy()

# Sort player wins by Player and Date
player_wins = player_wins.sort_values(['Player', 'Date'])

# Function to count wins in the last 30 days, excluding the current match
def wins_last_30_before_match(group):
    result = []
    for i in range(len(group)):
        cutoff = group.iloc[i]['Date']
        # Filter for wins that happened before the cutoff date within the last 30 days
        past_wins = group[(group['Date'] < cutoff) & (group['Date'] >= cutoff - pd.Timedelta(days=30))]
        result.append(len(past_wins))  # Count the wins in the past 30 days
    return pd.Series(result, index=group.index)

# Calculate wins in the last 30 days for each player before the match
player_wins['Wins_Last_30_Days_Before'] = (
    player_wins.groupby('Player', group_keys=False)
    .apply(wins_last_30_before_match)
)

# Merge the calculated win counts back into the original dataframe

# Sort for merge_asof (allows for the "before match" logic)
calc = calc.sort_values('Date')
player_wins = player_wins.sort_values('Date')

# Merge back for Fav (before the match date)
fav_win_counts = player_wins[['Player', 'Date', 'Wins_Last_30_Days_Before']].rename(columns={
    'Player': 'Fav',
    'Date': 'Match_Date',
    'Wins_Last_30_Days_Before': 'Fav_Wins_Last_30_Days_Before'
})

calc = pd.merge_asof(
    calc.sort_values('Date'),
    fav_win_counts.sort_values('Match_Date'),
    by='Fav',
    left_on='Date',
    right_on='Match_Date',
    direction='backward'
).drop(columns=['Match_Date'])

# Merge back for Dog (before the match date)
dog_win_counts = player_wins[['Player', 'Date', 'Wins_Last_30_Days_Before']].rename(columns={
    'Player': 'Dog',
    'Date': 'Match_Date',
    'Wins_Last_30_Days_Before': 'Dog_Wins_Last_30_Days_Before'
})

calc = pd.merge_asof(
    calc.sort_values('Date'),
    dog_win_counts.sort_values('Match_Date'),
    by='Dog',
    left_on='Date',
    right_on='Match_Date',
    direction='backward'
).drop(columns=['Match_Date'])

# Fill any missing win counts with 0 (if no prior wins)
calc['Fav_Wins_Last_30_Days_Before'] = calc['Fav_Wins_Last_30_Days_Before'].fillna(0).astype(int)
calc['Dog_Wins_Last_30_Days_Before'] = calc['Dog_Wins_Last_30_Days_Before'].fillna(0).astype(int)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_8172\74366135.py:33: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(wins_last_30_before_match)


In [25]:
calc['Predicted_Winner'] = calc['Fav_Wins_Last_30_Days_Before'] < calc['Dog_Wins_Last_30_Days_Before']
accuracy = (calc['Predicted_Winner'] == calc['Dog_Win']).mean()
print(f"Accuracy: {accuracy:.2%}")


Accuracy: 62.35%


In [30]:
stake=100
date_filter=calc[calc['Date']>'2021-07-01']
oddsfilter=date_filter[date_filter['Date']<'2025-07-05']
oddsfilter=oddsfilter[date_filter['Fav_Odds']>1.5].copy()
oddsfilter=oddsfilter[oddsfilter['Dog_Wins_Last_30_Days_Before']>oddsfilter['Fav_Wins_Last_30_Days_Before']]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Womens']
oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Dog_Odds'] - 1) * stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Dog']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter[['Date','Dog_Odds','Winner','Dog','Fav','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']].tail(5)
#oddsfilter.to_csv('_ClayResults_Grass.csv')

Total Profit: 9886.0 626 0.5319488817891374


C:\Users\Administrator\AppData\Local\Temp\ipykernel_8172\957194140.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  oddsfilter=oddsfilter[date_filter['Fav_Odds']>1.5].copy()


,Date,Dog_Odds,Winner,Dog,Fav,Dog_Wins_Last_30_Days_Before,Fav_Wins_Last_30_Days_Before
14738,2025-03-27,2.21,Maddison Inglis,Maddison Inglis,Erika Andreeva,3,1
14745,2025-03-27,2.23,Jaqueline Cristian,Ena Shibahara,Jaqueline Cristian,4,3
14760,2025-03-28,1.95,Rebeka Masarova,Rebeka Masarova,Bernarda Pera,5,2
14764,2025-03-29,2.08,Heather Watson,Heather Watson,Marina Stakusic,2,0
14773,2025-03-30,2.01,Jaqueline Cristian,Rebeka Masarova,Jaqueline Cristian,6,5


In [27]:
oddsfilter=date_filter[date_filter['Date']=='2025-07-05']
oddsfilter=oddsfilter[date_filter['Fav_Odds']>1.4].copy()
oddsfilter=oddsfilter[oddsfilter['Dog_Wins_Last_30_Days_Before']>oddsfilter['Fav_Wins_Last_30_Days_Before']]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Womens']
oddsfilter[['Fav','Dog','Fav_Wins_Last_30_Days_Before','Dog_Wins_Last_30_Days_Before','Winner']]

C:\Users\Administrator\AppData\Local\Temp\ipykernel_8172\3370333070.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  oddsfilter=oddsfilter[date_filter['Fav_Odds']>1.4].copy()


,Fav,Dog,Fav_Wins_Last_30_Days_Before,Dog_Wins_Last_30_Days_Before,Winner


In [31]:
stake=100
date_filter=calc[calc['Date']>'2021-01-01']
oddsfilter=date_filter[date_filter['Fav_Odds']>1.7].copy()
oddsfilter=oddsfilter[oddsfilter['Dog_Wins_Last_30_Days_Before']>oddsfilter['Fav_Wins_Last_30_Days_Before']]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Mens']
oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Dog_Odds'] - 1) * stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Dog']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter[['Date','Dog_Odds','Winner','Dog','Fav','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']]
#oddsfilter.to_csv('_GrassResults_Mens.csv')

Total Profit: 5640.0 677 0.5465288035450517


,Date,Dog_Odds,Winner,Dog,Fav,Dog_Wins_Last_30_Days_Before,Fav_Wins_Last_30_Days_Before
9,2023-07-10,1.97,Steve Johnson,Steve Johnson,Mikhail Kukushkin,3,1
17,2023-07-12,2.02,Tristan Schoolkate,Tristan Schoolkate,Altug Celikbilek,4,0
25,2023-07-14,1.89,Mark Lajal,Mark Lajal,Yasutaka Uchiyama,1,0
30,2023-07-14,2.07,Giovanni Mpetshi Perricard,Giovanni Mpetshi Perricard,Adam Walton,1,0
32,2023-07-15,2.08,Yuta Shimizu,Yuta Shimizu,Jason Jung,2,0
...,...,...,...,...,...,...,...
15141,2025-05-30,1.95,Rio Noguchi,Rio Noguchi,Paul Jubb,2,0
15148,2025-06-01,2.01,Michael Zheng,Michael Zheng,Liam Draxl,3,0
15166,2025-06-04,1.97,Andres Martin,Aziz Dougaz,Andres Martin,3,2
15200,2025-07-01,1.91,Daniil Glinka,Moerani Bouzige,Daniil Glinka,5,0


In [29]:
oddsfilter=date_filter[date_filter['Date']=='2025-07-05']
oddsfilter=oddsfilter[date_filter['Fav_Odds']>1.7].copy()
oddsfilter=oddsfilter[oddsfilter['Dog_Wins_Last_30_Days_Before']>oddsfilter['Fav_Wins_Last_30_Days_Before']]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Mens']
oddsfilter[['Fav','Dog','Fav_Wins_Last_30_Days_Before','Dog_Wins_Last_30_Days_Before','Winner']]

C:\Users\Administrator\AppData\Local\Temp\ipykernel_8172\461816032.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  oddsfilter=oddsfilter[date_filter['Fav_Odds']>1.7].copy()


,Fav,Dog,Fav_Wins_Last_30_Days_Before,Dog_Wins_Last_30_Days_Before,Winner
